### 一、图片收集

1、开启摄像头并显示其视频流

In [1]:
from FilesWeNeed.csi_camera import CSICamera
import ipywidgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg

#初始化摄像头
camera = CSICamera(width=224, height=224)
camera.running = True
camera.unobserve_all()#避免二次使用摄像头

#定义摄像头显示控件
camera_widget = ipywidgets.Image(format='jpeg')
def update_image(change):
    image = change['new']
    camera_widget.value = bgr8_to_jpeg(image)
camera.observe(update_image, names='value')
#显示摄像头控件
display(camera_widget)

Image(value=b'', format='jpeg')

In [1]:
#释放摄像头
camera.release_camera()

NameError: name 'camera' is not defined

2、遥控手柄初始化

In [24]:
import ipywidgets
from IPython.display import display
from jetracer.nvidia_racecar import NvidiaRacecar
import traitlets

#初始化遥控手柄
controller = ipywidgets.Controller(index=0)
#显示遥控手柄
display(controller)

#初始化小车
car = NvidiaRacecar()
car.throttle_gain = 0.14
car.steering_offset=0.11
car.steering_gain=0.75

Controller()

In [25]:
left_link = traitlets.dlink((controller.axes[0], 'value'), (car, 'steering'), transform=lambda x: -x)
right_link = traitlets.dlink((controller.axes[5], 'value'), (car, 'throttle'), transform=lambda x: -x)

In [2]:
import torchvision.transforms as transforms
from xy_dataset_addStop import XYDataset

TRANSFORMS = transforms.Compose([      
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#分为自动收集和手动收集两个数据集，需要更多不同场景时，需要重新添加另外搞个数据集，不能混在一起
TASK = 'racing'
CATEGORIES = ['apex24_弯道2增强']
DATASETS = ['AutoCollect', 'ManualCollect']
datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASK + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

#初始化数据集
dataset = datasets[DATASETS[0]]

3、定义传统的单张点击保存

In [3]:
import cv2
import ipywidgets 
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

#创建图片预览
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

#dataset_widget控件
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

#category_widget控件
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

#count_widget控件
count_widget = ipywidgets.IntText(description='count')
count_widget.value = dataset.get_count(category_widget.value)

#点击确定目标值以及保存图片
def save_snapshot(_, content, msg):
    print("点击有反应")
    if content['event'] == 'click': #点击触发事件函数
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
camera_widget.on_msg(save_snapshot)

#控件集合展示
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    dataset_widget,
    category_widget,
    count_widget
])
display(data_collection_widget)

4、连续图片收集

In [4]:
import time
# import winsound

#开启循环，约计7分钟每1000张
i = 0
while i < 200:
    i +=1
    #保存到不同文件夹时，修改下述函数第一个参数
    dataset.save_auto('apex24_弯道2增强', camera.value, count_widget.value)
    count_widget.value = dataset.get_count(category_widget.value)
    
    #每隔一段时间收集一次数据
    time.sleep(0.4)

### 二、图片批量处理

In [4]:
import cv2
import glob
import os
from jupyter_clickable_image_widget import ClickableImageWidget
import ipywidgets 
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import numpy as np
import uuid
import shutil

image_shot_widget = ClickableImageWidget(width=224, height=224)
snapshot_widget2 = ipywidgets.Image(width=224, height=224)
show_path_widget1 = ipywidgets.Text()
show_path_widget2 = ipywidgets.Text()

del_button1 = ipywidgets.Button(description='删除')
del_button2 = ipywidgets.Button(description='删除')
stop_button = ipywidgets.Button(description='停车')
straight_button = ipywidgets.Button(description='直走')
left_button = ipywidgets.Button(description='左转')
slow_button = ipywidgets.Button(description='减速')
speed_button = ipywidgets.Button(description='加速')

img = []
imagepath0={}
new_filename = ''
stop = 0
straight = 0
left = 0
slow = 0
speed = 0
count_new = 1

#定义迭代器，看文件名的计数在哪里
it = iter(sorted(glob.glob(os.path.join('racing_AutoCollect', 'apex8_中间','*.jpg')), 
                 key=lambda x: int((os.path.basename(x).split('.')[0]).split('_')[0])))

#点击删除按钮进行删除并转到下一张图片
def read_image(image_path):
    global img, imagepath0
    img = cv2.imread(image_path)
    image_shot_widget.value = bytes(bgr8_to_jpeg(img))
    imagepath0 = image_path
    show_path_widget1.value = imagepath0
    # print(imagepath0)

read_image(next(it))

#点击确定目标值以及保存图片
def save_snapshot2(_, content, msg):
    global new_filename,stop,straight,left,slow,speed,count_new

    if content['event'] == 'click': #点击触发事件函数
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']

        print(x,y)
        snapshot = cv2.circle(np.float32(img), (x, y), 8, (0, 255, 0), 3)
        snapshot_widget2.value = bgr8_to_jpeg(snapshot)
        
        #因为之前已经标注过，所以先不复制保留数据
        dest_dir = './racing_AutoCollect/已标注/Middle_8'
        shutil.copy(imagepath0,dest_dir)
        dest_file = os.path.join(dest_dir, os.path.basename(imagepath0).split('/')[-1])
        
        #重新改名
        new_filename = os.path.join(dest_dir, '%d_%d_%d_%d_%d_%d_%d_%d_%s.jpg' % (count_new,x,y,stop,straight,left,slow,speed,str(uuid.uuid1())))
        os.rename(dest_file,new_filename)
        stop = 0
        straight = 0
        left = 0
        slow = 0
        speed = 0
        count_new = count_new + 1
        show_path_widget2.value = new_filename
        read_image(next(it))
image_shot_widget.on_msg(save_snapshot2)

def stopIn():
    global stop
    stop=1
    return None
def straightIn():
    global straight
    straight=1
    return None
def leftIn():
    global left
    left=1
    return None
def slowIn():
    global slow
    slow=1
    return None
def speedIn():
    global speed
    speed=1
    return None

del_button1.on_click(lambda c: read_image(next(it)))     # 对于无用照片进行跳过
del_button2.on_click(lambda c: os.remove(new_filename)) # 对于确定错误目标值的照片进行移除
stop_button.on_click(lambda c: stopIn())
straight_button.on_click(lambda c: straightIn())
left_button.on_click(lambda c: leftIn())
slow_button.on_click(lambda c: slowIn())
speed_button.on_click(lambda c: speedIn())

all_widget = ipywidgets.HBox([
    ipywidgets.VBox([image_shot_widget,show_path_widget1,del_button1,stop_button,straight_button,left_button,slow_button,speed_button]), 
    ipywidgets.VBox([snapshot_widget2, show_path_widget2, del_button2]),
])

display(all_widget)

In [1]:
# import os
# import glob
# # print(glob.glob(os.path.join('racing_AutoCollect', 'apex', '*.jpg')))
# print(sorted(glob.glob(os.path.join('racing_AutoCollect', 'apex', '*.jpg')), key=lambda x: int((os.path.basename(x).split('.')[0]).split('_')[-1])))


### 三、添加路标标注

In [12]:
import cv2
import glob
import os
from jupyter_clickable_image_widget import ClickableImageWidget
import ipywidgets 
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import numpy as np
import uuid
import time

image_widget = ipywidgets.Image(width=224, height=224)

show_path_widget = ipywidgets.Text()
stop_button1 = ipywidgets.Button(description='停车')
stop_button2 = ipywidgets.Button(description='不停车')
img = []
new_filename = ''

#定义迭代器  按照图片编号进行排序
it = iter(sorted(glob.glob(os.path.join('racing_AutoCollect', 'processed_9_26_1800', '*.jpg')), 
                 key=lambda x: int((os.path.basename(x).split('.')[0]).split('_')[2])))

#首张图片显示
image_path0 = next(it)
# os.remove('racing_AutoCollect/processed_9_26_1800/127_58_4_e7fc0b72-1eaf-11ec-a343-845cf326e67a.jpg')
img = cv2.imread(image_path0)
image_widget.value = bytes(bgr8_to_jpeg(img))
new_filename = image_path0     #对首张图片不做处理

# for i in range(10):
#     img = cv2.imread(next(it))
#     image_widget.value = bytes(bgr8_to_jpeg(img))
#     time.sleep(1)

#点击删除按钮进行删除并转到下一张图片
def read_image(image_path, is_stop=0):
    global img, image_path0, new_filename
    
    #将上一个文件名更改                                 必须特定处理初始值和末尾值
    show_path_widget.value = image_path
    os.rename(image_path0,new_filename)
    
    img = cv2.imread(image_path)
    image_widget.value = bytes(bgr8_to_jpeg(img))
    
    #保留xy值
    x = int((os.path.basename(image_path).split('.')[0]).split('_')[0])
    y = int((os.path.basename(image_path).split('.')[0]).split('_')[1])
    count = (os.path.basename(image_path).split('.')[0]).split('_')[2]
    
    new_filename = './racing_AutoCollect/add_IsStop/%d_%d_%s_%s_%s.jpg' % (x, y, is_stop, count, str(uuid.uuid1()))
    image_path0 = image_path
    show_path_widget.value = new_filename

stop_button1.on_click(lambda c: read_image(next(it), 1))    
stop_button2.on_click(lambda c: read_image(next(it), 0)) 

all_widget = ipywidgets.HBox([
    ipywidgets.VBox([image_widget, show_path_widget, stop_button1, stop_button2]), 
])

display(all_widget)

### 文件处理

In [15]:
import os
import more_itertools
dest_dir = './racing_AutoCollect/test_5M_2R_1L_1S'

# for i in range(5):
path = './racing_AutoCollect/已标注/Stop_1'
it = iter(glob.glob(os.path.join(dest_dir, '*.jpg')))
for image in it:
    shutil.copy(image,dest_dir)

print(more_itertools.ilen(it))

ModuleNotFoundError: No module named 'more_itertools'